# MATH 405/607 

# Chaos In The Context Of Dynamical Systems

Callum Hepworth, Sandy Luo, Jed Yeo


## Content

1) Introduction to Chaos Theory
2) Strange Attractor & Lorenz Problem
3) Popular Examples of Chaos
4) Studying Chaos Numerically (ODEs)

## Introduction

- Chaos theory 
    - "Study of random or unpredictable behaviour in systems governed by deterministic laws" [1]
    - Example: Pinball machine
- Applications
    - Turbulent flow
    - Chemical reactions
    - Planetary motion

- Dynamical systems
    - Highly sensitive to initial conditions
    - i.e, adjusting initial conditions by some amount leads to interesting changes in the solution
    - Behavior of dynamical chaotic systems appears random
    - This is not the case - these systems are deterministic (future dynamics are fully defined by initial conditions)

## Lorenz Heat Problem

- Chaos first observed by Edward Lorenz in 1963 
- The problem: atmospheric convection (three ODEs):

$$\frac{dx}{dt} = \sigma(y-x), \frac{dy}{dt}=x(\rho-z)-y, \frac{dz}{dt}=xy-\beta z $$


- These three equations relate the properties of a 2D fluid layer uniformly warmed from below and cooled from above
- $x$ is proportional to rate of convection, $y$ to horizontal temperature gradient, $z$ to the vertical gradient
- Constants depend on physical properties of the system



In [1]:
# We take σ = 10, ρ = 28, β = 8/3
include("math405.jl")
using OrdinaryDiffEq, Plots
function lorenz(du, u, p, t)
    du[1] = 10.0*(u[2]-u[1])
    du[2] = u[1]*(28.0-u[3]) - u[2]
    du[3] = u[1]*u[2] - (8/3)*u[3]
end

lorenz (generic function with 1 method)

In [ ]:
u0 = [-15.0;-15.0;20.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz,u0,tspan)
sol0 = solve(prob, Tsit5())
plot(sol0, vars=(1,2,3))

┌ Warning: To maintain consistency with solution indexing, keyword argument vars will be removed in a future version. Please use keyword argument idxs instead.
│   caller = ip:0x0
└ @ Core :-1


In [ ]:
u1 = [-15.0;-15.0;20.0001]
prob = ODEProblem(lorenz,u1,tspan)
sol1 = solve(prob, Tsit5())
plot(sol1, vars=(1,2,3))

- These butterfly-like shapes reveal the structure of the "stranger attractor" for the Lorenz problem
- A mere adjustment of $1\times10^{-4}$ in the third initial condition yields a vastly different solution space

- There isn't a universally accepted definition of chaos, but in general a chaotic problem must loosely follow these guidelines:
   - 1) be sensitive to initial conditions
   - 2) be topologically transitive
   - 3) have dense periodic orbits
- It can be shown that the last two imply the first; we will not discuss these conditons as we are mainly interested in pertubations of the initial condition(s).

In [ ]:
# @gif for m = 1:50
#     u0 = [-15.0;-15.0;20]
#     tspan = (0.0,m*2.0)
#     prob = ODEProblem(lorenz,u0,tspan)
#     sol0 = solve(prob, Tsit5())
#     plot(sol0,vars=(1,3))
# end

In [ ]:
# @gif for m = 1:50
#     u1 = [-15.0;-15.0;20.0001]
#     tspan = (0.0,m*2.0)
#     prob = ODEProblem(lorenz,u1,tspan)
#     sol1 = solve(prob, Tsit5())
#     plot(sol1,vars=(1,3),color="orange")
# end

In [ ]:
plot(sol0, vars=(1,3), xlabel=L"x", ylabel=L"z", label="\$ z(0)=20 \$", title="\$ xz\$ plane with differing ICs")
plot!(sol1, vars=(1,3), label="\$ z(0)=20.0001 \$")

## Strange Attractors

- Classical mechanics
    - Dynamical system's behavior can be described as motion on an “attractor".

In [ ]:
z = 1

## Popular Examples of Chaos

i love examples

- Rossler equations
- Chaotic nonlinear forced operator

In [ ]:
example = 0

## Studying Chaos Numerically

- The process of solving chaotic problems numerically is no different from that of a regular ODE system
- e.g. the Lorenz problem is simply a system of 3 IVPs

$$\frac{dx}{dt} = \sigma(y-x), \frac{dy}{dt}=x(\rho-z)-y, \frac{dz}{dt}=xy-\beta z $$

- The process is similar to solving any of the other ODEs we've solved in class
- Throughout the presentation we opted to use the $\texttt{Tsit5}$ method to solve our systems
- Let's try our own implementation of Kutta third-order method and compare it to Improved Euler

- Butcher tableau:
$$\begin{array}{c|ccc} 
    0 &   &   &   \\ 
    1/2 & 1/2 &   &   \\ 
    1 & -1 & 2 &     \\ 
    \hline 
      & 1/6 & 2/3  & 1/6 
\end{array}$$ 

-  Improved Euler method
$$
    U_{n+1} = U_n + h \frac{f(t_n, U_n) + f\big(t_{n+1}, U_n + h f(t_n, U_n)\big)}{2}
$$ 
- $O(h^2)$ accuracy, 2 function evaluations per step

In [ ]:
function kutta3(f, u0, h, Tf)
    t = 0.0:h:Tf 
    # make sure that h and Tf are compatible!
    @assert t[end] ≈ Tf     
    U = zeros(length(u0), length(t))
    U[:, 1] = u0 
    for tn = 1:length(t)-1
        k1 = h * f(t[tn], U[:, tn])
        k2 = h * f(t[tn] + 0.5 * h, U[:, tn] + 0.5 * k1)
        k3 = h * f(t[tn] + 1 * h, U[:, tn] - 1 * k1 + 2 * k2)
        U[:, tn+1] = U[:, tn] + (1/6)*k1 + (2/3)*k2 + (1/6)*k3
    end
    return U, t 
end

function ieuler(f, u0, h, T)
    t = 0.0:h:T 
    U = zeros(length(u0), length(t)); U[:, 1] = u0 
    for n = 2:length(t) 
        F = f(t[n-1], U[:,n-1])
        U[:,n] = U[:,n-1] + 0.5 * h * (F + f(t[n], U[:,n-1] + h * F))
    end 
    return U, t
end

In [ ]:
f(t, u) = [10.0*(u[2]-u[1]), u[1]*(28.0-u[3]) - u[2], u[1]*u[2] - (8/3)*u[3]]
f0 = [-15.0, -15.0, 20.0]
rksoln, t_rk = kutta3(f, f0, 0.005, 30.0)
ieulersoln, t_e = ieuler(f, f0, 0.005, 30.0)
    
plot(t_rk, rksoln[1,:], xlabel="\$ t\$", ylabel="\$ x(t)", title="RK3 vs. Euler for Lorenz Problem \$x(t), h=0.0005\$", label="RK3")
plot!(t_e, ieulersoln[1,:], label="Improved Euler")

- The solutions appear to diverge from each other around 7 time steps
- Let's vary our step size and evaluate the value of the solution after a lengthy period of time

In [ ]:
times = 500:50:2000
final_rk = []
final_ie = []

for time in times
    rksoln, t_rk = kutta3(f, f0, 1/time, 1000.0)
    ieulersoln, t_e = ieuler(f, f0, 1/time, 1000.0)
    append!(final_rk, rksoln[1, end])
    append!(final_ie, ieulersoln[1, end])
end

hs = []
for time in times
    append!(hs, 1/time)
end

scatter(hs, final_rk, title="Value of \$x(t)\$ at \$t=1000.0\$ vs. \$h\$", xlabel=L"h", ylabel="\$x(t)\$", label="\$x(t)\$ w/ RK3")
scatter!(hs, final_ie, label="\$x(t)\$ w/ IE")

- Using both the third order Runge-Kutta method and improved Euler, there appears to be no correlation between the step size and the "final value" (i.e. solution value after some time period)
- Going to smaller step sizes doesn't guarantee uniform convergence
- Due to the butterfly-effect and floating point operations, numerical approaches cannot give a reliable long-term simulation of chaotic dynamic systems [2]
- This is simply an implication of using numerics to evaluate chaotic problems

# References

- [1] - zzzz
- [2] - https://www.researchgate.net/publication/318488232_Influence_of_round-off_errors_on_the_reliability_of_numerical_simulations_of_chaotic_dynamic_systems